<a href="https://colab.research.google.com/github/SheikhMudassarHanif/NLP/blob/main/testingsavedmodels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [16]:
#run this
import pandas as pd
import numpy as np
excel_file_path = '/content/preprocessed_data.csv'
column_names = pd.read_csv(excel_file_path, nrows=0).columns.tolist()
df = pd.read_csv(excel_file_path,usecols=[column_names[0],column_names[1]])
df.dropna(inplace=True)
df.head()

,SENTENCES,MEANING
0,how can I commun with my parent,م اپ والدین سے کیسا با کرنا
1,how can I make friend,م دوس کیسا بنائ
2,whi do I get so sad,م اتنا اداس کم ہ
3,if youv ask yourself such question your not alon,اگر اپ اپ اپ سے ایسا سوالا کرنا ہ ت اپ اکیلے نہ ہ
4,depend on where youv turn for guidanc you may ...,میں با پرنا منحصر ہونا کہنا اپ رہنمائ کم لینا ...


In [17]:
#dont run
train,val,test=np.split(df.sample(frac=1),[int(0.8*len(df)),int(0.9*len(df))])

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [18]:
vocab_size=7000
embedding_dim=50
max_length=500
trunc_type='post'
oov_tok='<OOV>'
test_inputs=test['SENTENCES '].values
test_inputs_str=[str(x) for x in test_inputs]
tokenizer=Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(test_inputs_str) # Fit the tokenizer on the text data first
test_sequences = tokenizer.texts_to_sequences(test_inputs_str)

# Filter out None values from the sequences
test_sequences = [[x for x in seq if x is not None] for seq in test_sequences]


test_padded = pad_sequences(test_sequences, maxlen=max_length, truncating=trunc_type)

test_labels=test['MEANING'].values
test_labels = [str(x) for x in test_labels]
tokenizer.fit_on_texts(test_labels)
test_labels_sequences = tokenizer.texts_to_sequences(test_labels)
padded_test_labels = pad_sequences(test_labels_sequences, maxlen=max_length, truncating=trunc_type)

In [19]:
test

,SENTENCES,MEANING
12673,but he go out and begin to publish it much and...,لیکن میں باہر جاکر بہ چرچا کرنا لگنا اور میں ب...
7270,take I just one visit to make their tenderloin...,میں کم ٹینڈرلوئن اسٹیک طوفان کو کراچ م اسٹیک پ...
29204,good food,اچھا کھا
3990,I have not hear much about it,م میں کم بارے م زیادہ نہ سننا ہونا ۔
12395,behold I have tell you befor,دیکھ م پہلا ہ میں سے کہہ دیا ہونا ۔
...,...,...
6884,it wa an okay experi a bit surpris to see an i...,میں ایک ٹھیک تجربہ تھا مینہ م جد دیکھ کرنا قدر...
11755,and everi one that heareth these say of mine a...,اور جو کوئ میر میں بات سن اور میں پرنا عمل نہ ...
8759,nihari have a hair strand in it and serious it...,نہار کم میں م بال ک پٹنا تھ اور سنجیدگ سے میں ...
6351,becaus their special peri peri bite not avail ...,"کیونکہ میں ک خصوصی ""پیر پیر بائٹ"" اج دستیاب نہ..."


Testing lstm model

In [20]:
model=tf.keras.models.load_model('/content/best_model2.h5lstm.keras')

In [21]:
model.evaluate(test_padded,padded_test_labels)

93/93 ━━━━━━━━━━━━━━━━━━━━ 268s 3s/step - accuracy: 0.9704 - loss: 0.2709


[0.26863566040992737, 0.9704049825668335]

In [24]:
new_data = ["I have "]
new_data = [str(x) for x in new_data]
# new_data=new_data.lower()
# new_data=Normalize_Input(new_data)


# The fit_on_texts method doesn't return anything, it updates the tokenizer's vocabulary.
# To get the sequences, use texts_to_sequences instead.
tokenizer.fit_on_texts(new_data) # Update the tokenizer's vocabulary
new_sequences = tokenizer.texts_to_sequences(new_data) # Get the sequences

new_padded = tf.keras.preprocessing.sequence.pad_sequences(new_sequences, maxlen=max_length, truncating=trunc_type)

# Make predictions
predictions = model.predict(new_padded)

# View the prediction result
print(predictions)
predicted_sequence = np.argmax(predictions, axis=-1)

# Convert the predicted sequence of indices back to words
predicted_sentence = [tokenizer.index_word.get(index, '<OOV>') for index in predicted_sequence[0]]

# Join the words to form a sentence
predicted_sentence_str = ' '.join(predicted_sentence)
print(predicted_sentence_str)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
[[[9.7178704e-01 5.1757012e-04 4.7108650e-04 ... 2.4844252e-07
   6.8496213e-08 7.1557309e-08]
  [9.9343282e-01 1.5048731e-04 1.2959623e-04 ... 2.7707662e-08
   6.5372969e-09 6.8750876e-09]
  [9.9480224e-01 1.2270655e-04 1.0468037e-04 ... 1.9371688e-08
   4.4580424e-09 4.6936295e-09]
  ...
  [9.9515629e-01 1.1507214e-04 9.7730517e-05 ... 1.7499010e-08
   4.0041330e-09 4.2176316e-09]
  [1.4878084e-01 2.2541448e-02 3.4761854e-02 ... 1.0766289e-05
   3.1756845e-06 3.1998561e-06]
  [5.4001097e-02 2.7591057e-02 4.8084669e-02 ... 1.1496065e-05
   3.3758295e-06 3.3741203e-06]]]
<OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> 